In [2]:
import zipfile
import os


def extract_zip(zip_file, output_dir):
    base = '/home/jems/cmsc422/p3'
    zip_file = os.path.join(base, zip_file)
    output_dir = os.path.join(base, output_dir)
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:

        # Create output directory if it doesn't exist
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        zip_ref.extractall(output_dir)

extract_zip('Face Data for Homework.zip', 'face_data') 

In [11]:
from PIL import Image
import numpy as np
from collections import defaultdict


image_dir = '/home/jems/cmsc422/p3/face_data/ATT'

grouped_images = defaultdict(list)

# Iterate over each file in the directory
for filename in os.listdir(image_dir):
    if filename.endswith('.png'):  # Ensure you're only looking at PNG files
        # Split the filename at the underscore and extract the first number
        number_1 = filename.split('_')[0]
        
        # If the number_1 is not already a key in the dictionary, add it with an empty list
        image_path = os.path.join(image_dir, filename)
        image = Image.open(image_path)
        image_arr = np.array(image).flatten()
        
        
        # Append the image data (not just the filename)
        grouped_images[number_1].append(image_arr)
# Now 'grouped_files' contains the dictionary of lists
print(grouped_images)

defaultdict(<class 'list'>, {'2': [array([ 38,  35,  36, ..., 138, 142, 141], dtype=uint8), array([34, 36, 35, ..., 32, 25, 25], dtype=uint8), array([ 43,  39,  36, ..., 154, 155, 141], dtype=uint8), array([35, 32, 36, ..., 25, 28, 29], dtype=uint8), array([ 35,  36,  37, ..., 144, 147, 143], dtype=uint8), array([ 34,  39,  35, ..., 135, 138, 148], dtype=uint8), array([ 37,  34,  38, ..., 135, 143, 133], dtype=uint8), array([30, 37, 35, ..., 26, 27, 32], dtype=uint8), array([34, 35, 35, ..., 31, 24, 24], dtype=uint8), array([37, 35, 35, ..., 26, 28, 28], dtype=uint8)], '36': [array([46, 41, 45, ..., 37, 34, 43], dtype=uint8), array([ 48,  47,  47, ..., 120, 104,  66], dtype=uint8), array([62, 64, 64, ..., 45, 44, 52], dtype=uint8), array([46, 48, 41, ..., 35, 36, 37], dtype=uint8), array([47, 42, 53, ..., 36, 70, 82], dtype=uint8), array([45, 45, 48, ..., 76, 79, 70], dtype=uint8), array([46, 45, 47, ..., 46, 50, 50], dtype=uint8), array([48, 42, 52, ..., 46, 51, 48], dtype=uint8), arr

In [16]:
print(len(grouped_images))

folds = [[] for i in range(5)]
classes = [[] for i in range(5)]
boundaries = [0, 2, 4, 6, 8, 10]

for i in range(1, 41):
    key = str(i)
    
    # If the key is in the dictionary, add the list of filenames to the appropriate fold
    if key in grouped_images:
        collection = grouped_images[key]
        for j in range(1, 6):
            folds[j-1].extend(collection[boundaries[j-1]:boundaries[j]])
            classes[j-1].extend([i]*2)


40


In [31]:
results = []
for i in range(5):
    test = folds[i]
    test_classes = classes[i]
    train = [folds[j] for j in range(5) if j != i]
    train = [item for sublist in train for item in sublist]
    train_classes = [classes[j] for j in range(5) if j != i]
    train_classes = [item for sublist in train_classes for item in sublist]
    print(len(test), len(train))
    
    for arr in test:
        closest = float('inf')
        closest_class = None
        for j in range(len(train)):
            trarr = train[j]
            # print(trarr)
            curr_class = train_classes[j]
            dist = np.linalg.norm(arr - trarr)
            if dist < closest:
                closest = dist
                closest_class = curr_class
        results.append((arr, closest_class))

        

print(results)

80 320
80 320
80 320
80 320
80 320
[(array([48, 49, 45, ..., 47, 46, 46], dtype=uint8), 34), (array([64, 76, 80, ..., 35, 37, 39], dtype=uint8), 22), (array([ 38,  35,  36, ..., 138, 142, 141], dtype=uint8), 34), (array([34, 36, 35, ..., 32, 25, 25], dtype=uint8), 34), (array([101, 106, 102, ...,  46,  50,  45], dtype=uint8), 39), (array([105, 102, 108, ...,  42,  35,  37], dtype=uint8), 39), (array([118, 114, 117, ...,  58,  54,  52], dtype=uint8), 22), (array([115, 113, 113, ...,  56,  52,  55], dtype=uint8), 22), (array([132, 127, 128, ...,  85,  83,  85], dtype=uint8), 21), (array([128, 128, 128, ...,  87,  83,  86], dtype=uint8), 21), (array([143, 144, 142, ...,  79,  79,  80], dtype=uint8), 29), (array([155, 154, 154, ..., 111, 111, 108], dtype=uint8), 29), (array([68, 68, 69, ..., 73, 67, 69], dtype=uint8), 39), (array([66, 68, 66, ..., 72, 72, 76], dtype=uint8), 39), (array([131, 128, 126, ...,  41,  40,  35], dtype=uint8), 39), (array([126, 130, 131, ...,  43,  51,  48], dtype

In [34]:

def check_accuracy(results):
    correct = 0
    total = 0
    for i in range(len(results)):
        arr = results[i][0]
        correct_class = None
        for key in grouped_images:
            for thing in grouped_images[key]:
                if np.array_equal(thing, arr):
                    correct_class = int(key)
                break

        if results[i][1] == correct_class:
            correct += 1
        total += 1
        print(results[i][1], correct_class)
    return correct/total

print(check_accuracy(results))

34 1
22 None
34 2
34 None
39 3
39 None
22 4
22 None
21 5
21 None
29 6
29 None
39 7
39 None
39 8
39 None
39 9
39 None
39 10
31 None
34 11
34 None
22 12
22 None
39 13
39 None
39 14
39 None
39 15
34 None
34 16
34 None
22 17
39 None
39 18
29 None
34 19
39 None
39 20
39 None
39 21
39 None
39 22
10 None
39 23
39 None
39 24
39 None
22 25
39 None
39 26
39 None
34 27
34 None
39 28
39 None
39 29
39 None
39 30
39 None
39 31
39 None
34 32
34 None
39 33
39 None
39 34
39 None
22 35
39 None
34 36
39 None
39 37
39 None
39 38
39 None
11 39
39 None
39 40
39 None
22 None
34 None
34 None
34 None
39 None
39 None
22 None
22 None
21 None
39 None
29 None
29 None
22 None
14 None
39 None
39 None
39 None
22 None
14 None
39 None
34 None
34 None
22 None
39 None
39 None
22 None
34 None
34 None
34 None
34 None
16 None
34 None
22 None
39 None
39 None
21 None
34 None
34 None
22 None
14 None
39 None
39 None
22 None
39 None
39 None
39 None
39 None
39 None
39 None
22 None
39 None
22 None
34 None
34 None
34 None
34 None
3